In [5]:
# Etapa 1: Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler


In [6]:
# Etapa 2: Carregamento do arquivo CSV em um DataFrame

df = pd.read_csv('campeonato-brasileiro-full.csv')
df = pd.DataFrame() 


In [7]:
# Etapa 3: Visualização das primeiras linhas do DataFrame (head)
display(df.head())


""


In [14]:
# Etapa 5: Estatísticas descritivas básicas (describe)
print(df.describe())


ValueError: Cannot describe a DataFrame without columns

In [10]:
# Etapa 6: Contagem de valores ausentes por coluna
print(df.isnull().sum())


Series([], dtype: float64)


In [ ]:
# Etapa 7: Tratamento de valores ausentes
    
df['arena'].fillna('Desconhecido', inplace=True)
print(df.isnull().sum())

In [9]:
# Etapa 8: Identificação de outliers (Z-Score em mandante_placar e visitante_placar)
df['z_score_mandante'] = np.abs(stats.zscore(df['mandante_placar']))
df['z_score_visitante'] = np.abs(stats.zscore(df['visitante_placar']))

# Um limiar comum para identificar outliers com Z-score é 3.
threshold = 3
outliers = df[(df['z_score_mandante'] > threshold) | (df['z_score_visitante'] > threshold)]
print(f"Encontrados {len(outliers)} jogos considerados outliers com Z-score > {threshold}")
display(outliers[['data', 'mandante', 'visitante', 'mandante_placar', 'visitante_placar']].head())



KeyError: 'mandante_placar'

In [ ]:
# Etapa 9: Remoção ou tratamento dos outliers identificados
print(f"Tamanho original do DataFrame: {df.shape}")
df_sem_outliers = df[(df['z_score_mandante'] <= threshold) & (df['z_score_visitante'] <= threshold)]
print(f"Tamanho do DataFrame sem outliers: {df_sem_outliers.shape}")
    
# Para o resto do script, vamos usar o DataFrame sem outliers e remover as colunas de Z-score
df = df_sem_outliers.drop(['z_score_mandante', 'z_score_visitante'], axis=1).copy()

In [ ]:
# Etapa 10: Normalização de uma coluna numérica (Min-Max Scaling)
scaler = MinMaxScaler()
df['rodada_normalizada'] = scaler.fit_transform(df[['rodada']])
display(df[['rodada', 'rodada_normalizada']].head())

In [ ]:
# Etapa 11: Codificação de variáveis categóricas (One-Hot Encoding)
df_encoded = pd.get_dummies(df, columns=['mandante_estado', 'visitante_estado'], prefix=['est_mand', 'est_vis'])
print("Formato do DataFrame após One-Hot Encoding:", df_encoded.shape)
print("Exemplo de novas colunas criadas:")
print(list(df_encoded.columns[-5:])) # Mostra as últimas 5 colunas

In [ ]:
# Etapa 12: Criação de matriz de correlação
colunas_corr = ['rodada', 'mandante_placar', 'visitante_placar']
matriz_correlacao = df[colunas_corr].corr()
display(matriz_correlacao)

In [ ]:
# Etapa 13: Gráfico de dispersão (placar mandante vs. placar visitante)
sns.scatterplot(data=df, x='mandante_placar', y='visitante_placar', alpha=0.6)
plt.title('Placar Mandante vs. Placar Visitante', fontsize=16)
plt.xlabel('Gols do Mandante')
plt.ylabel('Gols do Visitante')
plt.show()

In [ ]:
# Etapa 14: Filtragem de dados com base em uma condição
jogos_flamengo_mandante = df[df['mandante'] == 'Flamengo']
print(f"Encontrados {len(jogos_flamengo_mandante)} jogos do Flamengo como mandante.")
display(jogos_flamengo_mandante.head())

In [ ]:
# Etapa 15: Exportação do DataFrame processado para CSV
df.to_csv('campeonato_brasileiro_processado.csv', index=False)

In [ ]:
# Etapa 16: Conversão da coluna de datas e extração de novas colunas

df['data'] = pd.to_datetime(df['data'], errors='coerce')
df.dropna(subset=['data'], inplace=True) 

df['ano'] = df['data'].dt.year
df['mes'] = df['data'].dt.month
display(df[['data', 'ano', 'mes']].head())




In [ ]:
# Etapa 17: Agrupamento dos dados (número de vitórias por time)

vitorias = df[df['vencedor'] != '-']
contagem_vitorias = vitorias['vencedor'].value_counts()
print("Contagem de vitórias por time (Top 10):")
display(contagem_vitorias.head(10))




In [ ]:
# Etapa 18: Criação de colunas derivadas (diferença de gols)
print("\n### 18. Criando coluna derivada 'diferenca_gols' ###")
df['diferenca_gols'] = df['mandante_placar'] - df['visitante_placar']
display(df[['mandante', 'visitante', 'mandante_placar', 'visitante_placar', 'diferenca_gols']].head())




In [ ]:
# Etapa 19: Visualização com gráficos de barras (número de vitórias por time)
print("\n### 19. Gráfico de barras com número de vitórias por time ###")
top_15_vitorias = contagem_vitorias.head(15)
    
sns.barplot(x=top_15_vitorias.values, y=top_15_vitorias.index, palette='viridis', orient='h')
plt.title('Top 15 Times com Mais Vitórias no Campeonato Brasileiro', fontsize=16)
plt.xlabel('Número de Vitórias')
plt.ylabel('Time')
plt.show()




In [ ]:
# Etapa 20: Heatmap da correlação para visualizar relações de forma mais clara

plt.figure(figsize=(8, 6))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Heatmap de Correlação entre Rodada e Placares', fontsize=16)
plt.show()